### **검색 알고리즘 구현 코드**

##### **활용 데이터**
* 카카오맵 맛집 크롤링 데이터 : kakao_food_deduplicated.csv 


* 촬영지 데이터(미디어타입 별 분할) :
    * drama : media_drama.csv
    * movie : media_movie.csv
    * artist : media_artist.csv

* 숙박 데이터 : 
    * hotel_pre.csv

##### **알고리즘 구조 설계**
* 카카오맵 맛집 데이터와 촬영지 데이터 기준 각 drama, movie, artist가 다른 DB에 저장되어있다고 가정 

1) drama/movie/artist 중 하나의 분야에 가서 배우명/아티스트명 검색 
2) 지역 14개중에 선택 -> area = ['충청', '대구', '대전', '강원', '경기', '인천', '제주', '전라', '세종', '서울', '울산', '부산', '경상', '광주'] -> 각 데이터의 '지역명' 컬럼에 저장되어 있음 
2) 해당 배우나 아티스트가 촬영한 데이터를 선택한 지역에 해당하는 media_drama, media_movie, media_artist에서 추출 -> 장소타입별 3개씩만 추출!
3) 추출된 촬영지 데이터 근처 여행코스와 숙박 추천 -> 최종 추천 목록 : 맛집 3개, 카페 3개, 관광지(playground) 3개, 숙박 3개 
3) - (1) 촬영지 데이터에서 추출된 데이터의 카테고리가 "playground"로만 이루어져있다면, 카카오맵 맛집 데이터에서 맛집 3개, 카페 3개, 숙박 데이터에서 숙박 3개 추천
3) - (2) 예를 들어 촬영지 데이터에서 추출된 데이터의 카테고리가 "restaurant" 2개, "cafe" 1개, "playground" 3개로 이루어져있다면, 카카오맵 맛집 데이터에서 맛집 1개, 카페 2개, 숙박 데이터 3개 추천해서 항상 최종 추천 수를 12개로 맞추는 걸로 !
4) 추천된 코스 지도 시각화 

##### 데이터 불러오기 

In [ ]:
# 카카오맵 맛집 크롤링 데이터 
import pandas as pd

kakao_food = pd.read_csv("C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/카카오맵 맛집 크롤링_전처리_데이터/kakao_food_geocode.csv", encoding="utf-8-sig")
kakao_food

: 

In [ ]:
# 촬영지 데이터 : 드라마
media_drama = pd.read_csv(f"C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/촬영지 데이터/미디어유형별_전처리_데이터/media_drama_pre.csv", encoding='utf-8-sig')
media_drama

In [ ]:
# 촬영지 데이터 : 영화 
media_movie = pd.read_csv(f"C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/촬영지 데이터/미디어유형별_전처리_데이터/media_movie_pre.csv", encoding='utf-8-sig')
media_movie

In [ ]:
# 촬영지 데이터 : 아티스트 
media_artist = pd.read_csv(f"C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/촬영지 데이터/미디어유형별_전처리_데이터/media_artist_pre.csv", encoding='utf-8-sig')
media_artist

In [ ]:
# 숙박 데이터 
hotel = pd.read_csv(f"C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/숙박 데이터/hotel_pre.csv", encoding='utf-8-sig')
hotel

In [ ]:
hotel.rename(columns={
    'Unnamed: 0': '번호',
    'LDGS_NM': '숙박명',
    'LDGS_ADDR': '지역명',
    'LDGS_ROAD_NM_ADDR': '도로명주소',
    'GSRM_SCALE_CN': '규모',
    'LDGS_GRAD_VALUE': '성급',
    'LDGMNT_TY_NM': '숙박유형',
    'LDGS_AVRG_PRC': '평균가격',
    'LDGS_MIN_PRC': '최소가격',
    'LDGS_MXMM_PRC': '최대가격',
    'LDGS_AVRG_SCORE_CO': '평균평점',
    '위도': '위도',
    '경도': '경도'
}, inplace=True)
hotel.columns

* 추천 알고리즘 구현 
    * 카카오맵 지도의 웹사이트 링크는 저장된 html을 열고, 클릭해야 해당 링크로 들어가짐

In [ ]:
# import pandas as pd
# import folium

# from IPython.display import display

# # ---------------------- [1. 데이터 불러오기 및 전처리] ----------------------
# media_drama = pd.read_csv(f"C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/촬영지 데이터/미디어유형별_전처리_데이터/media_drama_pre.csv", encoding='utf-8-sig')
# media_movie = pd.read_csv(f"C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/촬영지 데이터/미디어유형별_전처리_데이터/media_movie_pre.csv", encoding='utf-8-sig')
# media_artist = pd.read_csv(f"C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/촬영지 데이터/미디어유형별_전처리_데이터/media_artist_pre.csv", encoding='utf-8-sig')
# kakao_food = pd.read_csv("C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/카카오맵 맛집 크롤링_전처리_데이터/kakao_food_geocode.csv", encoding="utf-8-sig")
# hotel_df = pd.read_csv(f"C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/숙박 데이터/hotel_pre.csv", encoding='utf-8-sig')

# hotel_df.columns = hotel_df.columns.str.strip()
# hotel_df.rename(columns={
#     'Unnamed: 0': '번호',
#     'LDGS_NM': '숙박명',
#     'LDGS_ADDR': '지역명',
#     'LDGS_ROAD_NM_ADDR': '도로명주소',
#     'GSRM_SCALE_CN': '규모',
#     'LDGS_GRAD_VALUE': '성급',
#     'LDGMNT_TY_NM': '숙박유형',
#     'LDGS_AVRG_PRC': '평균가격',
#     'LDGS_MIN_PRC': '최소가격',
#     'LDGS_MXMM_PRC': '최대가격',
#     'LDGS_AVRG_SCORE_CO': '평균평점',
# }, inplace=True)

# # ---------------------- [2. 사용자 입력] ----------------------
# media_type = input("검색할 분야 (drama, movie, artist): ").strip()
# name = input("배우 또는 아티스트 이름 입력: ").strip()
# region = input("지역 선택 (예: 경기): ").strip()

# # ---------------------- [3. 데이터 선택 및 필터링] ----------------------
# if media_type == 'drama':
#     df = media_drama
# elif media_type == 'movie':
#     df = media_movie
# elif media_type == 'artist':
#     df = media_artist
# else:
#     raise ValueError("media_type은 drama, movie, artist 중 하나여야 합니다.")

# # 이름, 지역 필터링
# if media_type in ['drama', 'movie']:
#     filtered = df[df['배우이름'].str.contains(name, na=False) & (df['주소_지역명'] == region)]
# else:
#     filtered = df[df['아티스트명'].str.contains(name, na=False) & (df['주소_지역명'] == region)]

# # ❗ 빈 경우 처리
# if filtered.empty:
#     print("해당 배우/아티스트와 지역 조합으로 데이터가 없습니다.")
#     m = folium.Map(location=[37.5665, 126.9780], zoom_start=10)  # 서울
#     display(m)
#     exit()

# # ---------------------- [4. 장소타입별 최대 3개 추출] ----------------------
# recommend_place = filtered.groupby('장소타입').apply(lambda x: x.head(3)).reset_index(drop=True)

# # 장소타입 보완을 위한 계산
# place_types = ['restaurant', 'cafe', 'playground']
# counts = recommend_place['장소타입'].value_counts().to_dict()
# recommend_count = {k: max(3 - counts.get(k, 0), 0) for k in place_types}

# # ---------------------- [5. 보완 장소 추천] ----------------------
# extra_restaurant = kakao_food[(kakao_food['카테고리'] == 'restaurant') & (kakao_food['지역명'] == region)].head(recommend_count['restaurant'])
# extra_cafe = kakao_food[(kakao_food['카테고리'] == 'cafe') & (kakao_food['지역명'] == region)].head(recommend_count['cafe'])

# # ---------------------- [6. 숙박 추천] ----------------------
# recommend_hotel = hotel_df[hotel_df['지역명'] == region].head(3)

# # ---------------------- [7. 지도 시각화] ----------------------
# # 위도, 경도 통합
# all_locations = pd.concat([
#     recommend_place[['위도', '경도']],
#     extra_restaurant[['위도', '경도']],
#     extra_cafe[['위도', '경도']],
#     recommend_hotel[['위도', '경도']]
# ], ignore_index=True)

# # 지도 중심
# center_lat = all_locations['위도'].mean()
# center_lon = all_locations['경도'].mean()
# m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# # 🎬 촬영지
# for _, row in recommend_place.iterrows():
#     popup = folium.Popup(f"""
#         <b>{row.get('제목', '제목 없음')}</b><br>
#         장소명: {row.get('장소명', '-') }<br>
#         장소타입: {row.get('장소타입', '-') }<br>
#         설명: {row.get('장소설명', '-') }<br>
#         주소: {row.get('주소', '-') }
#     """, max_width=300)
#     folium.Marker([row['위도'], row['경도']], popup=popup,
#                   icon=folium.Icon(icon='video-camera', prefix='fa', color='blue')).add_to(m)
    
# # 🍽 맛집/카페
# for _, row in pd.concat([extra_restaurant, extra_cafe]).iterrows():
#     리뷰수 = int(row.get('리뷰 수', 0))
#     website = row.get('웹사이트', '')

#     # # ✅ 여기가 print 넣을 위치
#     # print("웹사이트 링크 확인:", website)

#     if not pd.isna(website) and not website.startswith("http"):
#         website = "https://" + website

#     html = f"""
#         <b>{row.get('식당이름', '식당명 없음')}</b><br>
#         카테고리: {row.get('카테고리', '-') }<br>
#         별점: {row.get('별점', '-') }<br>
#         리뷰수: {리뷰수}<br>
#         주소: {row.get('주소', '-') }<br>
#         <a href="{website}" target="_blank">웹사이트</a>
#     """
#     popup = folium.Popup(folium.Html(html, script=True), max_width=300)

#     folium.Marker(
#         [row['위도'], row['경도']],
#         popup=popup,
#         icon=folium.Icon(icon='cutlery', prefix='fa', color='red')
#     ).add_to(m)

# # 🏨 숙박
# for _, row in recommend_hotel.iterrows():
#     popup = folium.Popup(f"""
#         <b>{row.get('숙박명', '이름 없음')}</b><br>
#         도로명 주소: {row.get('도로명주소', '-') }<br>
#         성급: {row.get('성급', '-') }<br>
#         최소가격: {row.get('최소가격', '-') }원<br>
#         최대가격: {row.get('최대가격', '-') }원<br>
#         평균평점: {row.get('평균평점', '-') }
#     """, max_width=300)
#     folium.Marker([row['위도'], row['경도']], popup=popup,
#                   icon=folium.Icon(icon='bed', prefix='fa', color='green')).add_to(m)

# # 지도 출력
# display(m)

# # 지도 저장
# m.save("C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/지도/recommendation_map_01.html")

#### ** 알고리즘 고도화**

* 거리 기반 추천 시스템으로 코드 수정 


    * geopy.distance.geodesic() 사용해서 거리 계산

    * 촬영지 좌표 평균값을 중심점으로 사용

    * 각 장소(restaurant, cafe, 숙박)에 대해 거리 계산 후 정렬하여 .head(3)



In [ ]:
import pandas as pd
import folium
from geopy.distance import geodesic
from IPython.display import display

# ---------------------- [1. 데이터 불러오기 및 전처리] ----------------------
media_drama = pd.read_csv("C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/촬영지 데이터/미디어유형별_전처리_데이터/media_drama_pre.csv", encoding='utf-8-sig')
media_movie = pd.read_csv("C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/촬영지 데이터/미디어유형별_전처리_데이터/media_movie_pre.csv", encoding='utf-8-sig')
media_artist = pd.read_csv("C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/촬영지 데이터/미디어유형별_전처리_데이터/media_artist_pre.csv", encoding='utf-8-sig')
kakao_food = pd.read_csv("C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/카카오맵 맛집 크롤링_전처리_데이터/kakao_food_geocode.csv", encoding="utf-8-sig")
hotel_df = pd.read_csv("C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/숙박 데이터/hotel_pre.csv", encoding='utf-8-sig')

hotel_df.columns = hotel_df.columns.str.strip()
hotel_df.rename(columns={
    'Unnamed: 0': '번호',
    'LDGS_NM': '숙박명',
    'LDGS_ADDR': '지역명',
    'LDGS_ROAD_NM_ADDR': '도로명주소',
    'GSRM_SCALE_CN': '규모',
    'LDGS_GRAD_VALUE': '성급',
    'LDGMNT_TY_NM': '숙박유형',
    'LDGS_AVRG_PRC': '평균가격',
    'LDGS_MIN_PRC': '최소가격',
    'LDGS_MXMM_PRC': '최대가격',
    'LDGS_AVRG_SCORE_CO': '평균평점',
}, inplace=True)

# ---------------------- [2. 사용자 입력] ----------------------
media_type = input("검색할 분야 (drama, movie, artist): ").strip()
name = input("배우 또는 아티스트 이름 입력: ").strip()
region = input("지역 선택 (예: 경기): ").strip()

# ---------------------- [3. 데이터 선택 및 필터링] ----------------------
if media_type == 'drama':
    df = media_drama
elif media_type == 'movie':
    df = media_movie
elif media_type == 'artist':
    df = media_artist
else:
    raise ValueError("media_type은 drama, movie, artist 중 하나여야 합니다.")

# 이름, 지역 필터링
if media_type in ['drama', 'movie']:
    filtered = df[df['배우이름'].str.contains(name, na=False) & (df['주소_지역명'] == region)]
else:
    filtered = df[df['아티스트명'].str.contains(name, na=False) & (df['주소_지역명'] == region)]

# ❗ 빈 경우 처리
if filtered.empty:
    print("해당 배우/아티스트와 지역 조합으로 데이터가 없습니다.")
    m = folium.Map(location=[37.5665, 126.9780], zoom_start=10)  # 서울
    display(m)
    exit()

# ---------------------- [4. 촬영지 정제 및 중심좌표 계산] ----------------------
recommend_place = filtered.groupby('장소타입').apply(lambda x: x.head(3)).reset_index(drop=True)

# ---------------------- [★ 선택형 추천 기능 추가 위치] ----------------------
# unique 장소 목록 출력용
display_df = recommend_place[['제목', '장소명', '장소타입']].drop_duplicates().reset_index(drop=True)

# 출력
print("🎬 추천 촬영지 목록 (1개만 선택):")
for i, row in display_df.iterrows():
    print(f"{i+1}. 제목: {row['제목']}, 장소: {row['장소명']}, 타입: {row['장소타입']}")

# 선택
selected_index = input(f"\n추천받고 싶은 촬영지 번호를 입력하세요 (1 ~ {len(display_df)}): ").strip()

# 정수 변환 및 유효성 검사
if selected_index.isdigit():
    selected_index = int(selected_index) - 1
    if 0 <= selected_index < len(display_df):
        selected_row = display_df.loc[selected_index]
        # 선택된 장소 기준으로 recommend_place 필터링
        recommend_place = recommend_place[
            (recommend_place['제목'] == selected_row['제목']) &
            (recommend_place['장소명'] == selected_row['장소명']) &
            (recommend_place['장소타입'] == selected_row['장소타입'])
        ].reset_index(drop=True)
    else:
        print("❗입력한 번호가 유효하지 않습니다.")
        exit()
else:
    print("❗숫자로 입력해주세요.")
    exit()

results = []
for idx, row in recommend_place.iterrows():
    place_name = row['장소명']
    place_lat = row['위도']
    place_lng = row['경도']
    place_type = row['장소타입']
    place_title = row.get('제목', '제목 없음')

    def calc_distance_to_place(r):
        try:
            return geodesic((place_lat, place_lng), (r['위도'], r['경도'])).meters
        except:
            return float('inf')

    kakao_food['거리'] = kakao_food.apply(calc_distance_to_place, axis=1)
    hotel_df['거리'] = hotel_df.apply(calc_distance_to_place, axis=1)

    closest_rests = kakao_food[(kakao_food['카테고리'] == 'restaurant') & (kakao_food['지역명'] == region)].sort_values('거리').head(3)
    closest_cafes = kakao_food[(kakao_food['카테고리'] == 'cafe') & (kakao_food['지역명'] == region)].sort_values('거리').head(3)
    closest_hotels = hotel_df[hotel_df['지역명'] == region].sort_values('거리').head(3)

    results.append({
        '촬영지 제목': place_title,
        '장소명': place_name,
        '장소타입': place_type,
        '맛집 목록': closest_rests['식당이름'].tolist() if not closest_rests.empty else ['-'],
        '맛집 거리(m)': [round(d, 2) for d in closest_rests['거리'].tolist()] if not closest_rests.empty else ['-'],
        '카페 목록': closest_cafes['식당이름'].tolist() if not closest_cafes.empty else ['-'],
        '카페 거리(m)': [round(d, 2) for d in closest_cafes['거리'].tolist()] if not closest_cafes.empty else ['-'],
        '숙소 목록': closest_hotels['숙박명'].tolist() if not closest_hotels.empty else ['-'],
        '숙소 거리(m)': [round(d, 2) for d in closest_hotels['거리'].tolist()] if not closest_hotels.empty else ['-'],
    })

recommend_detail = pd.DataFrame(results)
print("\n📍 각 촬영지별 주변 추천 (3개씩):")
display(recommend_detail)

In [ ]:
import pandas as pd

# 1) 맛집만 풀기
rows_rest = []
for _, row in recommend_detail.iterrows():
    places = row['맛집 목록']
    distances = row['맛집 거리(m)']
    for place, dist in zip(places, distances):
        rows_rest.append({
            '촬영지 제목': row['촬영지 제목'],
            '촬영지 장소명': row['장소명'],
            '맛집명': place,
            '거리(m)': dist
        })
extra_restaurant = pd.DataFrame(rows_rest)

# 2) 카페만 풀기
rows_cafe = []
for _, row in recommend_detail.iterrows():
    places = row['카페 목록']
    distances = row['카페 거리(m)']
    for place, dist in zip(places, distances):
        rows_cafe.append({
            '촬영지 제목': row['촬영지 제목'],
            '촬영지 장소명': row['장소명'],
            '카페명': place,
            '거리(m)': dist
        })
extra_cafe = pd.DataFrame(rows_cafe)

# 3) 숙소만 풀기
rows_hotel = []
for _, row in recommend_detail.iterrows():
    places = row['숙소 목록']
    distances = row['숙소 거리(m)']
    for place, dist in zip(places, distances):
        rows_hotel.append({
            '촬영지 제목': row['촬영지 제목'],
            '촬영지 장소명': row['장소명'],
            '숙소명': place,
            '거리(m)': dist
        })
recommend_hotel = pd.DataFrame(rows_hotel)

# 예: kakao_food와 extra_restaurant 조인
extra_restaurant = extra_restaurant.merge(
    kakao_food[['식당이름', '카테고리', '별점', '리뷰 수', '웹사이트', '주소', '위도', '경도']],
    left_on='맛집명', right_on='식당이름', how='left'
)

extra_cafe = extra_cafe.merge(
    kakao_food[['식당이름', '위도', '경도', '카테고리', '별점', '리뷰 수', '주소', '웹사이트']],
    left_on='카페명', right_on='식당이름', how='left'
)

recommend_hotel = recommend_hotel.merge(
    hotel_df[['숙박명', '도로명주소', '성급', '최소가격', '최대가격', '평균평점', '위도', '경도']],
    left_on='숙소명', right_on='숙박명', how='left'
)


In [ ]:
center_point = (recommend_place['위도'].mean(), recommend_place['경도'].mean())

# 지도 생성
m = folium.Map(location=center_point, zoom_start=12)

# 🎬 촬영지 마커
for _, row in recommend_place.iterrows():
    popup = folium.Popup(f"""
        <b>{row.get('제목', '제목 없음')}</b><br>
        장소명: {row.get('장소명', '-') }<br>
        장소타입: {row.get('장소타입', '-') }<br>
        설명: {row.get('장소설명', '-') }<br>
        주소: {row.get('주소', '-') }
    """, max_width=300)
    folium.Marker(
        [row['위도'], row['경도']], 
        popup=popup,
        icon=folium.Icon(icon='video-camera', prefix='fa', color='blue')
    ).add_to(m)

# 🍽 맛집 마커 (red, cutlery)
for _, row in extra_restaurant.iterrows():
    리뷰수 = int(row.get('리뷰 수', 0))
    website = row.get('웹사이트', '')
    if not pd.isna(website) and not website.startswith("http"):
        website = "https://" + website

    html = f"""
        <b>{row.get('식당이름', '식당명 없음')}</b><br>
        카테고리: {row.get('카테고리', '-') }<br>
        별점: {row.get('별점', '-') }<br>
        리뷰수: {리뷰수}<br>
        주소: {row.get('주소', '-') }<br>
        <a href="{website}" target="_blank">웹사이트</a>
    """
    popup = folium.Popup(folium.Html(html, script=True), max_width=300)

    folium.Marker(
        [row['위도'], row['경도']],
        popup=popup,
        icon=folium.Icon(icon='cutlery', prefix='fa', color='red')
    ).add_to(m)

# ☕ 카페 마커 (green, coffee)
for _, row in extra_cafe.iterrows():
    리뷰수 = int(row.get('리뷰 수', 0))
    website = row.get('웹사이트', '')
    if not pd.isna(website) and not website.startswith("http"):
        website = "https://" + website

    html = f"""
        <b>{row.get('식당이름', '카페명 없음')}</b><br>
        카테고리: {row.get('카테고리', '-') }<br>
        별점: {row.get('별점', '-') }<br>
        리뷰수: {리뷰수}<br>
        주소: {row.get('주소', '-') }<br>
        <a href="{website}" target="_blank">웹사이트</a>
    """
    popup = folium.Popup(folium.Html(html, script=True), max_width=300)

    folium.Marker(
        [row['위도'], row['경도']],
        popup=popup,
        icon=folium.Icon(icon='coffee', prefix='fa', color='orange')
    ).add_to(m)


# 🏨 숙박 마커
for _, row in recommend_hotel.iterrows():
    popup = folium.Popup(f"""
        <b>{row.get('숙박명', '이름 없음')}</b><br>
        도로명 주소: {row.get('도로명주소', '-') }<br>
        성급: {row.get('성급', '-') }<br>
        최소가격: {row.get('최소가격', '-') }원<br>
        최대가격: {row.get('최대가격', '-') }원<br>
        평균평점: {row.get('평균평점', '-') }
    """, max_width=300)
    folium.Marker(
        [row['위도'], row['경도']],
        popup=popup,
        icon=folium.Icon(icon='bed', prefix='fa', color='green')
    ).add_to(m)

# 지도 출력 및 저장
display(m)

save_path = "C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/지도/recommendation_map_re.html"
m.save(save_path)
print(f"지도가 저장되었습니다: {save_path}")
